In [328]:
import random
import numpy as np
import sympy as sp

In [329]:
L_VALUE=5

We first create a function to verify if a number is prime.

In [330]:
def VerifyPrime(num):
    if num == 1:
        print("Not a prime number")
        return False
    for i in range(2, num):
        if num % i == 0:
            print("Not a prime number")
            return False
    return True

In [331]:
print(VerifyPrime(1))
print(VerifyPrime(2))

Not a prime number
False
True


We then create a function that creates the subsets S1----SQ based on a set S of size L and Q the number of subets

In [332]:
set(range(3))

{0, 1, 2}

In [333]:
def generate_subset(L, Q, sparsity=0.01):
    subsets = []
    S = set(range(L))
    max_size = max(1,int(L * sparsity))  # Max size of each subset is ~1% of L
    for _ in range(Q):
        subset_size = random.randint(1, max_size)  # Random size between 1 and 1% of L
        subset = set(random.sample(S, subset_size))
        subsets.append(subset)
    return subsets

Demo of the function

We create a function to create the sparse matrix based on the subsets S1----SQ

In [334]:
def create_sparse_matrix(L,Q,subsets):
    M = np.zeros((L,Q),dtype=int) #In numpy rows then columns
    for j, subset in enumerate(subsets): #Because enumerate returns index of subset and value of subset
        for i in subset:
            M[i, j] = 1
    return M

We create the permutation function

In [335]:
sp.prime(2)

3

In [336]:
def hi_x(L,x,n):
    hi = [(1*x + 2) % L]
    for i in range(1,n):
        hi.append((sp.prime(i)*x + 2)% L)
    return hi

We will create a SIG Matrix based on the permutation function as well as the sparse matrix
We first create the permutation list h1 and h2

In [337]:
sparse_matrix = create_sparse_matrix(L_VALUE,4,generate_subset(L_VALUE,4))

/tmp/ipykernel_6177/3473781552.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset = set(random.sample(S, subset_size))


In [338]:
sparse_matrix.shape

(5, 4)

In [339]:
np.zeros((2,sparse_matrix.shape[1]),dtype=int)

array([[0, 0, 0, 0],
       [0, 0, 0, 0]])

In [340]:
h1_and_2 = [hi_x(L_VALUE,i,2) for i in range(L_VALUE)]
h1_and_2

[[2, 2], [3, 4], [4, 1], [0, 3], [1, 0]]

In [341]:
# signature_matrix = np.full((2,sparse_matrix.shape[1]),np.inf)
# for col in range(sparse_matrix.shape[1]):
#     for row in range(sparse_matrix.shape[0]):
#         if sparse_matrix[row, col] == 1:
#             for perm_idx in range(2):  # For each hash function
#                 hash_value = h1_and_2[row][perm_idx]
#                 signature_matrix[perm_idx, col] = min(signature_matrix[perm_idx, col], hash_value)

def signature_matrix(sparse_matrix, n_permutations,L):  
    num_columns = sparse_matrix.shape[1]
    signature_matrix = np.full((n_permutations, num_columns), np.inf)
    
    for col in range(num_columns):
        for row in range(sparse_matrix.shape[0]):
            if sparse_matrix[row, col] == 1:
                # Apply the permutation functions to the row index
                hash_values = hi_x(L, row, n_permutations)
                
                # Update the signature matrix with the minimum hash values
                for perm_idx in range(n_permutations):
                    signature_matrix[perm_idx, col] = min(signature_matrix[perm_idx, col], hash_values[perm_idx])
    
    return signature_matrix



Exemple from the classes

In [342]:
for row in range(2):
    print("azenaze")

azenaze
azenaze


In [343]:
#Permutation
def h1_class(L,X):
    h1 = [(1*i + 1) % 5 for i in X]
    return h1
def h2_class(L,X):
    h2 = [(3*i + 1) % 5 for i in X]
    return h2

#Sparse matrix
value = [1,0,1,0,0,1,1,1,0,0,1,1,1,0,0]
sparse_matrix_class = np.array(value).reshape(5,3)
print(sparse_matrix_class)


#Signature matrix
signature_matrix_class = np.full((2,sparse_matrix_class.shape[1]),np.inf)
print(signature_matrix_class)
for col in range(sparse_matrix_class.shape[1]):  # For each column (subset)
    row_indices = np.where(sparse_matrix_class[:, col] == 1)[0]  # Non-zero rows in the column
    
    # Apply both hash functions to the row indices
    h1_values = h1_class(L_VALUE, row_indices)
    h2_values = h2_class(L_VALUE, row_indices)
    
    # Update the signature matrix with the minimum hash values
    signature_matrix_class[0, col] = min(h1_values)  # First hash function
    signature_matrix_class[1, col] = min(h2_values)  # Second hash function

signature_matrix_class

[[1 0 1]
 [0 0 1]
 [1 1 0]
 [0 1 1]
 [1 0 0]]
[[inf inf inf]
 [inf inf inf]]


array([[0., 3., 1.],
       [1., 0., 0.]])

Jaccard scimilarity function

In [344]:
def approximate_jaccard_similarity(SIG, col1, col2):
    match_count = np.sum(SIG[:, col1] == SIG[:, col2])
    return match_count / SIG.shape[0]

approximate_jaccard_similarity(signature_matrix_class, 0, 2)


0.0

In [345]:
def actual_jaccard_similarity(M, pos_subset1, pos_subset2):
    subset_1 = set(np.where(M[:,pos_subset1] == 1)[0])
    subset_2 = set(np.where(M[:,pos_subset2] == 1)[0])
    intersection = len(subset_1 & subset_2)
    union = len(subset_1 | subset_2)
    return intersection / union

print(actual_jaccard_similarity(sparse_matrix_class,0,2))

0.2


With L big and Q varying

In [379]:
L_VALUE = 2749
Q_VALUE = 7
sparsity = 0.01
subsets = generate_subset(L_VALUE, Q_VALUE)
sparse_matrix = create_sparse_matrix(L_VALUE, Q_VALUE, subsets)
signature_matrix_ = signature_matrix(sparse_matrix, 2, L_VALUE)

for i in range(Q_VALUE):
    for j in range(i + 1, Q_VALUE):
        true_similarity = actual_jaccard_similarity(sparse_matrix, i, j)
        approx_similarity = approximate_jaccard_similarity(signature_matrix_, i, j)
        print(f"J(S_{i+1}, S_{j+1}) = {true_similarity:.4f}, JSIG(S_{i+1}, S_{j+1}) = {approx_similarity:.4f}")

J(S_1, S_2) = 0.0000, JSIG(S_1, S_2) = 0.0000
J(S_1, S_3) = 0.0000, JSIG(S_1, S_3) = 0.0000
J(S_1, S_4) = 0.0000, JSIG(S_1, S_4) = 0.0000
J(S_1, S_5) = 0.0000, JSIG(S_1, S_5) = 0.0000
J(S_1, S_6) = 0.0000, JSIG(S_1, S_6) = 0.0000
J(S_1, S_7) = 0.0000, JSIG(S_1, S_7) = 0.0000
J(S_2, S_3) = 0.0000, JSIG(S_2, S_3) = 0.0000
J(S_2, S_4) = 0.0000, JSIG(S_2, S_4) = 0.0000
J(S_2, S_5) = 0.0000, JSIG(S_2, S_5) = 0.0000
J(S_2, S_6) = 0.0000, JSIG(S_2, S_6) = 0.0000
J(S_2, S_7) = 0.0217, JSIG(S_2, S_7) = 0.5000
J(S_3, S_4) = 0.0000, JSIG(S_3, S_4) = 0.0000
J(S_3, S_5) = 0.0000, JSIG(S_3, S_5) = 0.0000
J(S_3, S_6) = 0.0000, JSIG(S_3, S_6) = 0.0000
J(S_3, S_7) = 0.0000, JSIG(S_3, S_7) = 0.0000
J(S_4, S_5) = 0.0238, JSIG(S_4, S_5) = 0.0000
J(S_4, S_6) = 0.0000, JSIG(S_4, S_6) = 0.0000
J(S_4, S_7) = 0.0000, JSIG(S_4, S_7) = 0.0000
J(S_5, S_6) = 0.0000, JSIG(S_5, S_6) = 0.0000
J(S_5, S_7) = 0.0000, JSIG(S_5, S_7) = 0.0000
J(S_6, S_7) = 0.0000, JSIG(S_6, S_7) = 0.0000


/tmp/ipykernel_6177/3473781552.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  subset = set(random.sample(S, subset_size))
